In [1]:
############### Import packages
import os, numpy as np, pandas as pd, time, glob, re, math
from tqdm import tqdm
from time import process_time

##########################################################
##################### parameter ##########################
##########################################################
obj_type = '10-Q'
data_type_text = 'text_data'
data_type_id = 'id_data'

############### Set working directory to parent directory
os.getcwd()
# os.chdir('F:\\github\\narrative_conservatism\\code')

'F:\\github\\narrative_conservatism\\code'

In [2]:
########################################################################################
############ Concatenate and prepare merge: ID_DATA and TEXT_DATA ######################
########################################################################################

############## Define a function to concatenate all csv files with file name that matches a certain pattern into one data frame
def concatenate (indir, file_name_match):
    os.chdir(indir)
    file_list = glob.glob(file_name_match)
    df_list = list()
    colnames = pd.read_csv(file_list[0], header = None).loc[0]
    
    for filename in file_list:
        # print(filename)
        df = pd.read_csv(filename, low_memory = False)
        df_list.append(df)

    df_concat = pd.concat(df_list, axis = 0)
    df_concat.columns = colnames
    return df_concat

############## Concatenate id_data and text_data files and create two data frames
id_data = concatenate('..\\filings', data_type_id + '_'+ obj_type + '_' + '*.csv')
text_data = concatenate('..\\filings', data_type_text + '_'+ obj_type + '_' + '*.csv')
############## Save id_data dataframe into local file id_data_10-Q.csv
id_data.to_csv('..\\filings\\' + data_type_id + '_'+ obj_type + '.csv', index = 0)

############## text_data modifications #####################
############## Calculate tone : (tone = n_pos - n_negation - n_neg)/nw
text_data = text_data.assign(tone = (text_data['n_pos'] - text_data['n_negation'] - text_data['n_neg'])/text_data['nw'])

############## Correct modal words labels in text_data
text_data.columns = ['accnum', 'nw', 'nvocab', 'n_neg', 'n_pos', 'n_uctt', 'n_lit', 'n_cstr', \
                     'n_modal_strong', 'n_modal_moderate', 'n_modal_weak', 'n_negation', 'tone']

############## Save text_data dataframe into local file text_data_10-Q.csv
text_data.to_csv('..\\filings\\' + data_type_text + '_'+ obj_type + '.csv', index = 0)

print('Number of 10-Qs in edgar from 1993 Q1 to 2020 Q1: ' + str(len(id_data.index)))
print('Number of 10-Qs parsed and downloaded: ' + str(len(text_data.index)))

Number of 10-Qs in edgar from 1993 Q1 to 2020 Q1: 594017
Number of 10-Qs parsed and downloaded: 575579


In [3]:
########################################################################################
############ Merge compustat quarterly data with CRSP monthly data #####################
########################################################################################

########### Read compustat raw data files
comp_cols = ['gvkey', 'datadate', 'fyearq', 'fqtr', 'fyr', 'cusip', 'conm', 'curcdq', 'actq', \
             'atq', 'ceqq', 'cheq', 'cshoq', 'dlcq', 'dlttq', 'dpq', 'ibq', 'intanq', 'lctq', 'revtq', 'txditcq', 'exchg', \
             'cik', 'costat', 'prccq', 'addzip', 'incorp', 'sic', 'ipodate']
comp = pd.read_csv('..\\filings\\compustat.csv', usecols = comp_cols)
comp.columns = ['gvkey', 'date', 'fyearq', 'fqtr', 'fyr', 'cusip', 'conm', 'curcdq', 'actq', \
             'atq', 'ceqq', 'cheq', 'cshoq', 'dlcq', 'dlttq', 'dpq', 'ibq', 'intanq', 'lctq', 'revtq', 'txditcq', 'exchg', \
             'cik', 'costat', 'prccq', 'addzip', 'incorp', 'sic', 'ipodate']

### Reassign compustat column order
# 1st line: merge keys
# 2nd line: extra id info
# 3rd line: financial data
comp = comp[['cusip', 'cik', 'date', \
             'gvkey', 'conm', 'sic', 'incorp', 'addzip', 'fyearq', 'fqtr', 'fyr', 'ipodate', 'costat', 'curcdq', 'exchg', \
             'actq', 'atq', 'ceqq', 'cheq', 'cshoq', 'dlcq', 'dlttq', 'dpq', 'ibq', 'intanq', 'lctq', 'revtq', 'txditcq', 'prccq']]

########### Read crsp raw data files
crsp_cols = ['date', 'CUSIP', 'RET', 'vwretd']
crsp = pd.read_csv('..\\filings\\crsp.csv', usecols = crsp_cols)
crsp.columns = ['date', 'cusip', 'ret', 'vwretd']

### Delete the two 'day digits' of compustat and crsp filings' datadate
comp['date'] = comp['date'].astype(str).str[:-2]
crsp['date'] = crsp['date'].astype(str).str[:-2]

### Delete the 9th digit of compustat filings' cusip, and filter filings that have 8-digits cusip after deletion 
comp['cusip'] = comp['cusip'].astype(str).str[:-1]
print('number of quarterly filings in Compustat: ' + str(len(comp.index)))
comp = comp.loc[comp['cusip'].str.len() == 8]
print('number of quarterly filings in Compustat after deleting non-9-digits cusips: ' + str(len(comp.index)))

### Delete crsp raw rows that contains non-numeric returns ('B' and 'C'), fill NA with 0
print('number of monthly data in CRSP: ' + str(len(crsp.index)))
crsp = crsp[(crsp['ret'] != 'B') & (crsp['ret'] != 'C')]
crsp['ret'] = crsp['ret'].fillna(0)
print('number of monthly data in CRSP that contains only numeric returns: ' + str(len(crsp.index)))

### Mutate adjusted monthly returns and delete ret and vwretd
crsp = crsp.assign(adj_ret_m = crsp['ret'].astype(float) - crsp['vwretd'].astype(float))
crsp = crsp.drop(columns=['ret', 'vwretd'])
crsp

number of quarterly filings in Compustat: 1240141
number of quarterly filings in Compustat after deleting non-9-digits cusips: 1239632
number of monthly data in CRSP: 2477547
number of monthly data in CRSP that contains only numeric returns: 2427687


,date,cusip,adj_ret_m
0,199211,36720410,-0.057287
1,199212,36720410,-0.032898
2,199301,36720410,-0.012644
3,199302,36720410,0.012455
4,199303,36720410,-0.014063
...,...,...,...
2477542,201908,88160R10,-0.045951
2477543,201909,88160R10,0.051615
2477544,201910,88160R10,0.288162
2477545,201911,88160R10,0.012724


In [4]:
##################### Left merge CRSP and Compstat, key not unique in Compustat
crsp_comp = pd.merge(crsp, comp, on = ['cusip', 'date'], how='left', validate = '1:m')
crsp_comp

,date,cusip,adj_ret_m,cik,gvkey,conm,sic,incorp,addzip,fyearq,...,cshoq,dlcq,dlttq,dpq,ibq,intanq,lctq,revtq,txditcq,prccq
0,199211,36720410,-0.057287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,199212,36720410,-0.032898,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,199301,36720410,-0.012644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,199302,36720410,0.012455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,199303,36720410,-0.014063,43350.0,12994.0,GAS NATURAL INC,4924.0,OH,44114,1993.0,...,1.085,3.61,8.55,0.347,1.260,0.0,7.66,11.718,2.493,14.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428674,201908,88160R10,-0.045951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2428675,201909,88160R10,0.051615,1318605.0,184996.0,TESLA INC,3711.0,DE,94304,2019.0,...,180.000,2253.00,12383.00,530.851,143.469,537.0,10146.00,6302.860,0.000,240.87
2428676,201910,88160R10,0.288162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2428677,201911,88160R10,0.012724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
########## Aggregate the monthly returns in CRSP to quarterly returns by summing up 3-months returns in a quarter
adj_ret = list()
gvkey = crsp_comp['gvkey'].values.tolist()
adj_ret_m = crsp_comp['adj_ret_m'].values.tolist()

for index, value in enumerate(gvkey):
    ### requires a quarter-end flag to be non-zero, and the quarter-end flag of previous two months to be zero (avoid time-slot imprecision)
    if math.isnan(value) == False and math.isnan(gvkey[index-1]) == True and math.isnan(gvkey[index-2]) == True:
        adj_ret_i = adj_ret_m[index] + adj_ret_m[index-1] + adj_ret_m[index-2]
    else:
        adj_ret_i = float('NaN')
    adj_ret.append(adj_ret_i)

crsp_comp = crsp_comp.assign(RET = adj_ret)

########## Delete adj_ret_m column, and any rows that contains missing values for gvkey and RET
crsp_comp = crsp_comp.drop(columns=['adj_ret_m'])
crsp_comp = crsp_comp[crsp_comp['gvkey'].notnull()]
crsp_comp = crsp_comp[crsp_comp['RET'].notnull()]

############## Save text_data dataframe into local file text_data_10-Q.csv
crsp_comp.to_csv('..\\filings\\crsp_comp_' + obj_type + '.csv', index = 0)

In [13]:
################# Inspect crsp_comp 
first_T1PB = len(crsp_comp.index)
print('number of quarterly fillings after merging CRSP and Compustat: ' + str(first_T1PB))
crsp_comp

number of quarterly fillings after merging CRSP and Compustat: 707333


,date,cusip,cik,gvkey,conm,sic,incorp,addzip,fyearq,fqtr,...,dlcq,dlttq,dpq,ibq,intanq,lctq,revtq,txditcq,prccq,RET
4,199303,36720410,43350.0,12994.0,GAS NATURAL INC,4924.0,OH,44114,1993.0,3.0,...,3.610,8.550,0.347,1.260,0.000,7.660,11.718,2.493,14.25,-0.014252
7,199306,36720410,43350.0,12994.0,GAS NATURAL INC,4924.0,OH,44114,1993.0,4.0,...,1.663,11.050,0.390,-0.296,0.000,4.881,4.058,2.763,16.50,0.154818
10,199309,36720410,43350.0,12994.0,GAS NATURAL INC,4924.0,OH,44114,1994.0,1.0,...,4.323,10.750,0.362,-0.320,0.000,6.753,3.978,3.309,16.75,-0.011296
13,199312,36720410,43350.0,12994.0,GAS NATURAL INC,4924.0,OH,44114,1994.0,2.0,...,4.423,10.768,0.366,0.621,0.000,10.299,9.172,3.323,18.50,0.086561
16,199403,36720410,43350.0,12994.0,GAS NATURAL INC,4924.0,OH,44114,1994.0,3.0,...,2.623,10.763,0.380,1.090,0.000,7.651,9.768,3.197,17.25,-0.013350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428666,201812,88160R10,1318605.0,184996.0,TESLA INC,3711.0,DE,94304,2018.0,4.0,...,2629.460,9454.055,496.737,139.483,350.651,9992.136,7225.873,0.000,332.80,0.407883
2428669,201903,88160R10,1318605.0,184996.0,TESLA INC,3711.0,DE,94304,2019.0,1.0,...,1914.073,10834.402,467.577,-702.135,347.880,9242.800,4541.464,0.000,279.86,-0.294656
2428672,201906,88160R10,1318605.0,184996.0,TESLA INC,3711.0,DE,94304,2019.0,2.0,...,2011.177,12309.747,578.572,-408.334,480.833,9588.773,6349.676,0.000,223.46,-0.208027
2428675,201909,88160R10,1318605.0,184996.0,TESLA INC,3711.0,DE,94304,2019.0,3.0,...,2253.000,12383.000,530.851,143.469,537.000,10146.000,6302.860,0.000,240.87,0.075005


In [7]:
########################################################################################
######################## Merge ID_DATA with CRSP_COMPUSTAT #############################
########################################################################################

#id_data = concatenate('..\\filings', data_type_id + '_'+ obj_type + '_' + '*.csv')

############## prepare merge: ID_DATA
id_data['fd'] = id_data['fd'].str.replace('-', '')
id_data['rp'] = id_data['rp'].str.replace('-', '')
id_data['date'] = id_data['rp'].astype(str).str[:-2]

In [8]:
############## Left merge ID_DATA and CRSP_COMP, key not unique in both data sets
id_crsp_comp = pd.merge(id_data, crsp_comp, on = ['cik', 'date'], how = 'inner', validate = 'm:m')

### Delete duplicated columns in ID_DATA and CRSP_COMP, duplicated rows in accnum, and rename sic
id_crsp_comp = id_crsp_comp.drop(columns=['conm', 'sic_x', 'date', 'item8k', 'bazip', 'file_type', 'fye', 'curcdq'])
id_crsp_comp = id_crsp_comp[id_crsp_comp.duplicated('accnum') == False]
id_crsp_comp = id_crsp_comp[id_crsp_comp.duplicated(subset=['cik', 'rp']) == False]
id_crsp_comp = id_crsp_comp.rename(columns={'sic_y': 'sic'})

### Reassign compustat column order
# 1st line: merge keys
# 2nd line: extra id info
# 3rd line: financial data
id_crsp_comp = id_crsp_comp[['cusip', 'cik', 'rp', 'accnum', \
            'name', 'gvkey', 'sic', 'fd', 'fyearq', 'fqtr', 'fyr', 'ipodate', 'incorp', 'state', 'addzip', 'costat', 'exchg', \
            'actq', 'atq', 'ceqq', 'cheq', 'cshoq', 'dlcq', 'dlttq', 'dpq', 'ibq', 'intanq', 'lctq', 'revtq', 'txditcq', 'prccq', 'RET']]

In [14]:
################## Inspect id_crsp_comp 
second_T1PB = len(id_crsp_comp.index)
print('number of observations after merging with edgar data: ' + str(second_T1PB))
id_crsp_comp

number of observations after merging with edgar data: 303843


,cusip,cik,rp,accnum,name,gvkey,sic,fd,fyearq,fqtr,...,dlcq,dlttq,dpq,ibq,intanq,lctq,revtq,txditcq,prccq,RET
0,54626810,60512,19930630,0000060512-94-000005,LOUISIANA LAND & EXPLORATION CO,6819.0,1311.0,19930813,1993.0,2.0,...,86.500,356.300,27.300,5.600,NaN,208.200,189.300,136.100,42.5000,-0.059108
1,88579Y10,66740,19930630,0000066740-94-000015,MINNESOTA MINING & MANUFACTURING CO,7435.0,2670.0,19930813,1993.0,2.0,...,829.000,679.000,264.000,331.000,NaN,3365.000,3540.000,0.000,108.0000,-0.014779
2,08750910,11860,19930930,0000011860-94-000005,BETHLEHEM STEEL CORP /DE/,2189.0,3312.0,19931112,1993.0,3.0,...,100.300,728.500,69.100,30.700,NaN,908.400,1055.300,0.000,14.3750,-0.270146
3,54626810,60512,19930930,0000060512-94-000007,LOUISIANA LAND & EXPLORATION CO,6819.0,1311.0,19931110,1993.0,3.0,...,0.000,747.700,28.200,-1.800,NaN,195.100,187.900,124.800,44.6250,0.025969
4,88579Y10,66740,19930930,0000066740-94-000016,MINNESOTA MINING & MANUFACTURING CO,7435.0,2670.0,19931112,1993.0,3.0,...,796.000,682.000,262.000,316.000,NaN,3404.000,3481.000,0.000,102.8750,-0.076728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307477,87182910,96021,20191228,0000096021-20-000017,SYSCO CORP,10247.0,5140.0,20200205,2020.0,2.0,...,897.619,8654.524,185.011,383.410,4879.128,6931.968,15025.042,142.301,85.5400,-0.001390
307478,87840910,96699,20191228,0001171843-20-000876,TECHNICAL COMMUNICATIONS CORP,10364.0,3663.0,20200211,2020.0,1.0,...,0.148,0.521,0.006,-0.480,0.000,0.574,0.666,0.000,5.0500,1.266332
307479,86737U10,96793,20191231,0001564590-20-004619,SUNLINK HEALTH SYSTEMS INC,10380.0,5912.0,20200213,2020.0,2.0,...,0.782,0.851,0.350,0.203,1.295,6.440,12.805,0.000,1.0700,-0.135859
307480,87288520,98338,20191130,0001213900-20-000888,TSR INC,10305.0,7371.0,20200113,2019.0,2.0,...,0.252,0.271,0.002,0.061,0.000,5.241,15.233,0.000,3.2001,-0.315755


In [10]:
############## Left merge ID_CRSP_COMP and TEXT_DATA, key unique in both data sets
id_crsp_comp_text = pd.merge(id_crsp_comp, text_data, on = ['accnum'], how = 'inner', validate = '1:1')
id_crsp_comp_text.to_csv('..\\filings\\id_crsp_comp_text_' + obj_type + '.csv', index = 0)

############## Drop files that contain number of words beyond 1% (and 99%) threshold
nwq01 = id_crsp_comp_text['nw'].quantile(.01)
print('number of words, 1% quantile: ' + str(nwq01))
# nwq99 = id_crsp_comp_text['nw'].quantile(.99)
# print('number of words, 99% quantile: ' + str(nwq99))
id_crsp_comp_text = id_crsp_comp_text.loc[id_crsp_comp_text['nw'] >= nwq01]

third_T1PB = str(len(id_crsp_comp_text.index))
print('Number of firm-quarters after dropping files that contain number of words less than 1% threshold: ' + str(third_T1PB))

In [12]:
############# Table 1 Panel A: Summary statistics for text data
T1PA = id_crsp_comp_text.describe().transpose() # generally consistent with LM's summary statistics
T1PA.to_excel('..\\output\\Tables.xlsx', sheet_name='T1PA', float_format="%.2f")

T1PA

,count,mean,std,min,25%,50%,75%,max
cik,300806.0,8.252419e+05,431217.673614,2.000000e+01,7.223130e+05,8.970770e+05,1.068292e+06,1.783400e+06
gvkey,300806.0,5.556105e+04,59534.060765,1.004000e+03,1.150600e+04,2.607100e+04,6.628800e+04,3.321150e+05
sic,300806.0,4.951720e+03,1961.905251,1.000000e+02,3.537000e+03,4.955000e+03,6.351000e+03,9.997000e+03
fyearq,300806.0,2.006470e+03,7.259995,1.993000e+03,2.000000e+03,2.006000e+03,2.013000e+03,2.020000e+03
fqtr,300775.0,2.008069e+00,0.816992,1.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00
fyr,300806.0,1.038676e+01,3.063814,1.000000e+00,1.000000e+01,1.200000e+01,1.200000e+01,1.200000e+01
ipodate,166792.0,1.997741e+07,74491.006475,1.968031e+07,1.993093e+07,1.996112e+07,2.000081e+07,2.019102e+07
exchg,300793.0,1.349754e+01,2.471665,0.000000e+00,1.100000e+01,1.400000e+01,1.400000e+01,2.100000e+01
actq,228135.0,9.331677e+02,4139.533926,0.000000e+00,3.801200e+01,1.298760e+02,4.650310e+02,1.676330e+05
atq,299687.0,6.658008e+03,59052.732796,0.000000e+00,1.072840e+02,4.744720e+02,1.961145e+03,3.293755e+06


In [15]:
########### Table 1 Panel B: Data screening process
print('Number of firm-quarters from Compustat/CRSP merged data, 1993Q1 to 2019Q4: ' + str(first_T1PB))
print('Number of firm-quarters after merging edgar data: ' + str(second_T1PB))
print('Number of firm-quarters after dropping files that contain number of words less than 1% threshold: ' + str(third_T1PB))

########## other measure of tone?
########## other proxy for news?
########## how to separate tone

Number of firm-quarters from Compustat/CRSP merged data, 1993Q1 to 2019Q4: 707333
Number of firm-quarters after merging edgar data: 303843
Number of firm-quarters after dropping files that contain number of words less than 1% threshold: 300806
